(dynamic_programming)=
# Dynamic Programming
``` {index} Dynamic Programming
```

Dynamic algorithms are a family of programs which (broadly speaking) share the feature of utilising solutions to subproblems is coming up with optimal solutions. We will discuss the conditions which problems need to satisfy in order to be solved dinamically but let us first have a look at some basic examples. The contcept is quite challenging to digest, so there are many examples in this section.

## Simple Examples

* **Dynamic Fibonnacci** If you have read the section about [recursion](https://primer-computational-mathematics.github.io/book/b_coding/Fundamentals%20of%20Computer%20Science/2_Recursion.html#recursion) you probably know that the basic inplementation for generating Fibonacci numbers is very inefficient (\\(O(2^n)\\)!). Now, as we generate next numbers in the Fibonacci sequence, we will memoize them for further use. This *trade-of between memory and time* is dramatic in this case. Compare the two versions of the function:

In [39]:
# For comparing the running times
import time

# Inefficient
def fib(n):
    assert n >= 0
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)
    
# Dynamic version
def dynamicFib(n):
    assert n >= 0
    #prepare a table for memoizing
    prevFib = 1
    prevPrevFib = 1
    temp = 0

    #build up on your previous results
    for i in range(2,n+1):
        temp = prevFib + prevPrevFib
        prevPrevFib = prevFib
        prevFib = temp
    return prevFib
    
start = time.time()
print(fib(32))
end = time.time()
print("Time for brute:" + str(end - start))

start = time.time()
print(dynamicFib(32))
end = time.time()
print("Time for dynamic:" + str(end - start))

3524578
Time for brute:0.815772533416748
3524578
Time for dynamic:0.0


The time difference is dramatic! As you can probably spot, `dynamicFib` is \\(O(n)\\)! With a use of three integer variables (`prevFib`, `prevPrevFib` and `temp`) we brought exponential time complexity down to linear. How did it happen? Let us now depict the work done by `fib(5)` on a graph:

```{figure} algo_images/FibTree.png
:width: 60%
```

Wow! This is quite a tree! However, the worst feature is that many of the nodes are repeated (e.g. node \\(2\\) appears 3 times). These repeated results which are constantly recomputed bring us to the exponential complexity. Consider the the dynamic solution graph:

```{figure} algo_images/FibDyn.png
:width: 10%
```

Now, the number of nodes grows linearly with \\(n\\). We have `merged` the subproblems to avoid redundancy.

* **Shortest Path on a Grid** Dynamic programming is often used for optimisation problems. Consider a square grid with numbers from 0 to 9 in each square. An example would be:

```
+-+-+-+-+
|1|0|8|4| 
+-+-+-+-+
|3|5|1|0|
+-+-+-+-+
|6|8|9|3|
+-+-+-+-+
|1|2|4|5|
+-+-+-+-+
```

It is allowed to move only **down or right**. What is the value of the minimum path from the upper-left corner to the lower-right corner?

The initial approach to the problem might be to check all the possible paths and return the minimal one. This is clearly exponetial and too slow. To come up with a faster approach we need to find *overlapping* subproblems (this feature is often called ***optimal substructure***).

Let us imagine that we have reached the lower-right corner (hurray!). We could get there from the tile above it and left of it. This might at first seem like a *Divide and Conquer* problem, but let us keep on looking for an overlap:

```{figure} algo_images/DownRightGrid.png
:width: 60%
```

In the simplest case of four tiles, we can already see that the upper-left tile is considered twice. We should then levrage this repetition. This overlapping generalises for larger grids. In our alorithm, we will remember (***memoize***) the optimal path to the tiles already visited and build on this knowledge:

In [23]:
# grid is a square matrix
def shortestPathDownRight(grid):
    #dictionary that will keep the optimal paths to the already visited tiles
    memo = {}
    # n - length of the side of the square
    n = len(grid)
    assert n == 0 or len(grid[0]) == n
    memo[(0,0)] = grid[0][0]
    # top and most left column have trival optimal paths
    for i in range(1, n):
        memo[(0,i)] = memo[(0,i-1)] + grid[0][i]
        memo[(i,0)] = memo[(i-1,0)] + grid[i][0]
    # the dynamic magic    
    for i in range(1,n):
        for j in range(1,n):
            memo[(i,j)] = min(memo[(i-1,j)],memo[(i,j-1)]) + grid[i][j]
            
    return memo[(n-1,n-1)]

grid = [[1,0,8,4],[3,5,1,0],[6,8,9,3],[1,2,4,5]]
print(shortestPathDownRight(grid))

15


What is the time complexity of this algorithm? Based on the nested loop we can deduce that it is \\(O(n^2)\\). Quite an inprovement!

### Space Complexity

We usually do not concern ourselves with the amount of memory a program utilises. Dynamic programs are an exception to this rule. The amount of memory they use can be a limiting factor for some machines, so we need to take it into account. In case of `dynamicFib` this was \\(O(1)\\) as we only needed to keep track of the last two Fibonacci numbers. In case of `shortestPathDownRight` we need do create a grid of size \\(n \times n\\), so \\(O(n^2)\\). The notion of space complexity is very similai to time complexity, so we will not discuss it in depth.



* **Cutting Rods** We are given a rod of integer length \\(n\\) and a list of lenght `prices` \\(n\\). The \\(i\\)th entry in the list correspods to the profit we can get from selling the rod of length \\(i\\). How should we cut the rod so we maximise our profit?
The key to our dynamic algorithm will be the observation that:

We cut the rod at position \\(k\\). Now we have a rod of lenght \\(k\\) and \\(n-k\\). Let us assume we know the maximum price for these two rods. Now we need to consider all the \\(0 \leq k \leq \lfloor \frac{n}{2} \rfloor + 1\\) (the problem is symmetric so computing for \\(\frac{n}{2} \leq k\\) would be redundant. For \\(k=0\\) we just take `prices[n]`. The cutting introduces subproblems which are smaller than the initial problem and they are overlapping! Let us put this into code:

In [44]:
# For comparing the running times
import time
# For testing
from random import randint

def dynamicCutRod(n,prices):
    #setting the initial values of variables
    assert n >= 0 and n == len(prices)
    # trival cases
    if n == 0:
        return 0
    if n == 1:
        return prices[0]
    
    # setting up needed variables
    memo = {}
    currLen = 2
    memo[0] = 0
    memo[1] = prices[0]
    while currLen < n + 1:
        # no cuts for a given len
        memo[currLen] = prices[currLen - 1]
        # considering all possible cuts for a give len
        for k in range(1,currLen//2 + 1):
            # take the maximal one
            if memo[currLen] < memo[k] + memo[currLen - k]:
                memo[currLen] = memo[k] + memo[currLen - k]     
        currLen += 1
    return memo[n]
        
# for testing purposes
def bruteForceRecursive(n,prices):
    assert n >=0
    if n == 0:
        return 0
    if n == 1:
        return prices[0]
    currLen = n
    res = prices[n-1]
    for k in range(1,n//2 + 1):
        res = max(res, bruteForceRecursive(k,prices) + bruteForceRecursive(n-k,prices))
    return res


for i in range(1, 11):
    prices = []
    for j in range(i):
        prices.append(randint(1,10))
        
    assert bruteForceRecursive(len(prices),prices) == dynamicCutRod(len(prices),prices)

prices = []
for i in range(20):
    prices.append(randint(1,10))
    
start = time.time()
print(bruteForceRecursive(len(prices),prices))
end = time.time()
print("Time for brute:" + str(end - start))

start = time.time()
print(dynamicCutRod(len(prices),prices))
end = time.time()
print("Time for dynamic:" + str(end - start))

200
Time for brute:0.2918405532836914
200
Time for dynamic:0.0


Time complexity? For each \\(0\leq i \leq n \\) we need to consider \\(i\\) different cuts. This is the sum of arithmetic series so \\(O(n^2)\\). We memoize only the optimal solutions to subproblems, therefore \\(O(n)\\).

## Characteristic features of dynamic problems

All problems which can be tackled with the use of Dynamic Programming (DP) have two main features:

1) Optimal substructure: In broad terms, that solution to the problem can be formulated in terms of independent subproblems. Then you choose solution/combination of optimal solutions to subproblems and show that this choice leads to an optimal global solution. For example, for the `shortestPath` problem we choose between the optimal solution to the tile above and tile to the left. For the rod problem, we choose between all possible \\(n\\) cuts of a rod and assume we have solutions to the shorter rods.  

2) Overlapping subproblems: The subproblems which we split the main question into should repeat. Eliminating the need to recompute the subproblems (by memoising them) should speed up out algorythm. For example, `dynamicFib` used two variables to keep the results of computing previous Fibonacci numbers.

## Common approaches to DP

There are two main schools of thought when it comes to solving DP algorithms:

1) ***Memoization*** is using the *memory-time trade-off*. It remembers the results to the subproblems in a dictionary `memo` and recalls them when needed. If we have the result in memory, we take if from there, otherwise compute it.

2) ***Bottom-up*** approach starts with the trivial cases of the problem and builds on them. I relies on the the fact that subproblems can be sorted. So we first compute the optimal cutting for a rod of length 1 and then length 2 and so on...
